# Front matter

In [1]:
import requests
import json
import pandas as pd

In [2]:
catalog_api_output = json.loads(open("../data/portal_contents_raw.json").read())

In [3]:
json_endpoint_output = requests.get("https://data.cityofnewyork.us/data.json").json()

In [4]:
catalog_endpoints = [d['permalink'].split("/")[-1] for d in catalog_api_output]

In [5]:
json_endpoints = [json_endpoint_output['dataset'][i]['landingPage'].split("/")[-1] for i in 
                  range(len(json_endpoint_output['dataset']))]

In [6]:
set(json_endpoint_output).issubset(catalog_endpoints)

False

In [12]:
[endpoint for endpoint in json_endpoint_output if endpoint not in catalog_endpoints]
# These appear to just be new datasets.
# I generated the catalog list a week ago now.

['conformsTo', '@context', 'dataset', '@id', 'describedBy', '@type']

# The match

The idea here is to check what kinds of content the JSON endpoint emits by matching those endpoints against the endpoints returned by a whole-catalog search in the catalog API.

In [8]:
json_endpoint_types = []
for i, endpoint in enumerate(json_endpoints):
    try:
        catalog_ind = catalog_endpoints.index(json_endpoints[i])
    except ValueError:
        json_endpoint_types.append("new")
    else:
        json_endpoint_types.append(catalog_api_output[catalog_ind]['resource']['type'])

In [9]:
pd.Series(json_endpoint_types).value_counts()

dataset    1062
href        174
map         158
file         91
new          37
story         1
dtype: int64

Everything is neat and tidy.

# Saving the data

In [26]:
datasets = []
for i, endpoint in enumerate(json_endpoints):
    try:
        catalog_ind = catalog_endpoints.index(json_endpoints[i])
    except ValueError:
        pass
    else:
        datasets.append(catalog_api_output[catalog_ind])

Certain recently uploaded datasets are missing because they don't match because the JSON and Catalog data were generated at slightly different times, and it furthermore appears that there is a little bit of lag between them.

In [27]:
len(datasets)

1486

In [28]:
datasets[0]

{'classification': {'categories': [],
  'domain_category': 'Environment',
  'domain_metadata': [{'key': 'Update_Automation', 'value': 'Yes'},
   {'key': 'Update_Update-Frequency', 'value': 'As needed'},
   {'key': 'Dataset-Information_Agency',
    'value': 'Department of Environmental Protection (DEP)'}],
  'domain_tags': [],
  'tags': []},
 'link': 'https://data.cityofnewyork.us/Environment/CATS-Permits/f4rp-2kvy',
 'metadata': {'domain': 'data.cityofnewyork.us'},
 'permalink': 'https://data.cityofnewyork.us/d/f4rp-2kvy',
 'resource': {'attribution': 'Department of Environmental Protection (DEP)',
  'columns_description': ['',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   ''],
  'columns_field_name': ['requestid',
   'applicationid',
   'requesttype',
   'house',
   'street',
   'borough',
   'bin',
   'block',
   'lot',
   'ownername',
   'expirationdate',
   'make',
   'model',
   'burnermake',

In [29]:
with open("../data/datasets.json", "w") as f:
    f.write(json.dumps(datasets, indent=4))